## 練習時間
參考 Day 12 範例程式，離散化你覺得有興趣的欄位，並嘗試找出有趣的訊息

In [1]:
# Import 需要的套件
import os
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

### 之前做過的處理

In [2]:
# 設定 data_path
dir_data = './data/'
f_app_train = os.path.join(dir_data, 'application_train.csv')
f_app_test = os.path.join(dir_data, 'application_test.csv')

app_train = pd.read_csv(f_app_train)
app_test = pd.read_csv(f_app_test)

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

# Create an anomalous flag column
app_train['DAYS_EMPLOYED_ANOM'] = app_train["DAYS_EMPLOYED"] == 365243
app_train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
# also apply to testing dataset
app_test['DAYS_EMPLOYED_ANOM'] = app_test["DAYS_EMPLOYED"] == 365243
app_test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)

# absolute the value of DAYS_BIRTH
app_train['DAYS_BIRTH'] = abs(app_train['DAYS_BIRTH'])
app_test['DAYS_BIRTH'] = abs(app_test['DAYS_BIRTH'])

In [3]:
app_train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,DAYS_EMPLOYED_ANOM
0,100002,1,0,0,1,0,202500.0,406597.5,24700.5,351000.0,...,0,0,0,0,0,1,0,1,0,False
1,100003,0,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,...,1,0,0,0,0,0,0,1,0,False
2,100004,0,1,1,1,0,67500.0,135000.0,6750.0,135000.0,...,0,0,0,0,0,0,0,0,0,False
3,100006,0,0,0,1,0,135000.0,312682.5,29686.5,297000.0,...,0,0,0,0,0,0,0,0,0,False
4,100007,0,0,0,1,0,121500.0,513000.0,21865.5,513000.0,...,0,0,0,0,0,0,0,0,0,False


In [4]:
app_train_new = app_train[['TARGET','DAYS_BIRTH']]
app_train_new.head()

,TARGET,DAYS_BIRTH
0,1,9461
1,0,16765
2,0,19046
3,0,19005
4,0,19932


In [5]:
app_train_new['YEAR_BIRTH'] = app_train['DAYS_BIRTH']/365

app_train_new.head()

C:\ProgramData\Anaconda3\envs\tensorflow_cpu\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,TARGET,DAYS_BIRTH,YEAR_BIRTH
0,1,9461,25.920548
1,0,16765,45.931507
2,0,19046,52.180822
3,0,19005,52.068493
4,0,19932,54.608219


In [6]:
train= pd.qcut(app_train_new['YEAR_BIRTH'], 10)

In [7]:
train.head(10)

0    (20.517, 28.175]
1    (43.151, 47.178]
2     (51.74, 56.093]
3     (51.74, 56.093]
4     (51.74, 56.093]
5    (43.151, 47.178]
6      (36.0, 39.496]
7     (47.178, 51.74]
8     (51.74, 56.093]
9    (39.496, 43.151]
Name: YEAR_BIRTH, dtype: category
Categories (10, interval[float64]): [(20.517, 28.175] < (28.175, 32.038] < (32.038, 36.0] < (36.0, 39.496] ... (47.178, 51.74] < (51.74, 56.093] < (56.093, 60.77] < (60.77, 69.121]]

In [8]:
train.value_counts()

(43.151, 47.178]    30764
(28.175, 32.038]    30760
(36.0, 39.496]      30759
(51.74, 56.093]     30758
(56.093, 60.77]     30757
(20.517, 28.175]    30752
(39.496, 43.151]    30745
(32.038, 36.0]      30745
(60.77, 69.121]     30738
(47.178, 51.74]     30733
Name: YEAR_BIRTH, dtype: int64

In [9]:
train = pd.cut(app_train_new['YEAR_BIRTH'], [0,10,20,30,40,50,60,70,80,90])
train.value_counts()

(30, 40]    82308
(40, 50]    76541
(50, 60]    68062
(20, 30]    45021
(60, 70]    35579
(80, 90]        0
(70, 80]        0
(10, 20]        0
(0, 10]         0
Name: YEAR_BIRTH, dtype: int64

https://ithelp.ithome.com.tw/articles/10194003     Pandas資料的取得與篩選

In [10]:
train = app_train_new[app_train_new['TARGET']==1]

In [11]:
train.head()

,TARGET,DAYS_BIRTH,YEAR_BIRTH
0,1,9461,25.920548
26,1,18724,51.298630
40,1,17482,47.895890
42,1,13384,36.668493
81,1,24794,67.928767


In [12]:
train = pd.qcut(train['YEAR_BIRTH'], 10)
train.head()

0     (21.035, 27.097]
26     (46.95, 52.096]
40     (46.95, 52.096]
42    (35.959, 39.129]
81    (57.899, 68.953]
Name: YEAR_BIRTH, dtype: category
Categories (10, interval[float64]): [(21.035, 27.097] < (27.097, 29.791] < (29.791, 32.707] < (32.707, 35.959] ... (42.769, 46.95] < (46.95, 52.096] < (52.096, 57.899] < (57.899, 68.953]]

In [13]:
train.value_counts()

(52.096, 57.899]    2483
(46.95, 52.096]     2483
(35.959, 39.129]    2483
(32.707, 35.959]    2483
(29.791, 32.707]    2483
(21.035, 27.097]    2483
(57.899, 68.953]    2482
(42.769, 46.95]     2482
(27.097, 29.791]    2482
(39.129, 42.769]    2481
Name: YEAR_BIRTH, dtype: int64

In [14]:
train = app_train_new[app_train_new['TARGET']==1]
train = pd.cut(train['YEAR_BIRTH'], [0,10,20,30,40,50,60,70,80,90])
train.value_counts()

(30, 40]    7888
(40, 50]    5856
(20, 30]    5158
(50, 60]    4172
(60, 70]    1751
(80, 90]       0
(70, 80]       0
(10, 20]       0
(0, 10]        0
Name: YEAR_BIRTH, dtype: int64

In [15]:
train = app_train_new[app_train_new['TARGET']==0]
train = pd.cut(train['YEAR_BIRTH'], [0,10,20,30,40,50,60,70,80,90])
train.value_counts()

(30, 40]    74420
(40, 50]    70685
(50, 60]    63890
(20, 30]    39863
(60, 70]    33828
(80, 90]        0
(70, 80]        0
(10, 20]        0
(0, 10]         0
Name: YEAR_BIRTH, dtype: int64